Imports:

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
from torchvision import transforms
import matplotlib.image as mpimg
from skimage import measure
import json
import matplotlib.pyplot as plt
import time
import torch
import torch.nn as nn
import torch.optim as optim
import os
from sklearn.metrics import classification_report, confusion_matrix
from tqdm import tqdm

In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("Using GPU for training.")
else:
    device = torch.device("cpu")
    print("No GPU available, using CPU for training.")

Using GPU for training.


In [ ]:
%run "../DL_notebooks/src/dataloading.ipynb"
%run "../DL_notebooks/src/training.ipynb"
%run "../DL_notebooks/src/evaluate.ipynb"
%run "../DL_notebooks/src/visualizations.ipynb"
%run "../DL_notebooks/model_architectures.ipynb"

Data loaded and split into training, validation and testing datasets


Model Making for classification:

In [ ]:
# GradCAM initialization
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.model.eval()
        self.features_map = None
        self.grad = None
        self.hook = self.register_hooks()

    def register_hooks(self):
        def forward_hook(module, input, output):
            self.features_map = output

        def backward_hook(module, grad_in, grad_out):
            self.grad = grad_out[0]

        return [
            self.target_layer.register_forward_hook(forward_hook),
            self.target_layer.register_backward_hook(backward_hook),
        ]

    def generate(self, input_tensor, class_idx):
        self.model.zero_grad()
        one_hot = torch.zeros_like(outputs)
        one_hot[:, class_idx] = 1.0

        # Forward pass
        outputs = self.model(input_tensor)
        target = torch.sum(one_hot * outputs)

        # Backward pass
        target.backward(retain_graph=True)

        # Grad-CAM calculation
        weights = torch.mean(self.grad, dim=(2, 3), keepdim=True)
        gradcam_map = torch.sum(weights * self.features_map, dim=1, keepdim=True)
        gradcam_map = nn.functional.relu(gradcam_map)

        return gradcam_map

Classification Training:

In [ ]:
model = MRI_ResNet2().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# GradCAM initialization
gradcam_layer = model.model.layer4[-1]  # Adjust the layer based on your model architecture
gradcam = GradCAM(model, target_layer=gradcam_layer)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    print("Epoch ", epoch)
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix

# Function to overlay GradCAM map on input images
def overlay_gradcam(original_images, gradcam_map):
    alpha = 0.5  # Set the transparency of the overlay

    for i in range(original_images.shape[0]):
        img = original_images[i].numpy().transpose((1, 2, 0))  # Assuming images are in the shape (C, H, W)
        gradcam_img = gradcam_map[i, 0].detach().cpu().numpy()

        # Normalize GradCAM map to [0, 1]
        gradcam_img = (gradcam_img - np.min(gradcam_img)) / (np.max(gradcam_img) - np.min(gradcam_img) + 1e-8)

        # Overlay GradCAM map on the original image
        overlaid_img = alpha * gradcam_img + (1 - alpha) * img

        # Display the original image, GradCAM map, and overlaid image
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 3, 1)
        plt.imshow(img)
        plt.title('Original Image')

        plt.subplot(1, 3, 2)
        plt.imshow(gradcam_img, cmap='jet')
        plt.title('GradCAM Map')

        plt.subplot(1, 3, 3)
        plt.imshow(overlaid_img)
        plt.title('Overlaid Image')
        
        plt.show()
# Inference and visualization using Grad-CAM
model.eval()
all_preds = []
all_labels = []

for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    optimizer.zero_grad()
    outputs = model(inputs)
    
    # Compute the loss
    loss = criterion(outputs, labels)

    # Use GradCAM to visualize tumor localization
    gradcam_map = gradcam.generate(inputs, class_idx=0)  # Adjust class_idx based on your classes

    # Overlay gradcam_map on the input images for visualization
    overlay_gradcam(inputs, gradcam_map)

    # Save predictions and labels for evaluation
    _, preds = torch.max(outputs, 1)
    all_preds.extend(preds.cpu().numpy())
    all_labels.extend(labels.cpu().numpy())

# Classification Accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Classification Accuracy: {accuracy * 100:.2f}%")

# Confusion Matrix
conf_matrix = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(conf_matrix)

Epoch  0


C:\Users\gabeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


KeyboardInterrupt: 

Classification Testing:

Tumor Localization:

Visualization:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms, datasets
from torch.utils.data import DataLoader
from gradcam import GradCAM  # Implement GradCAM for PyTorch model

class MRI_ClassificationModel(nn.Module):
    def __init__(self, num_classes):
        super(MRI_ClassificationModel, self).__init__()
        self.resnet = models.resnet152(pretrained=True)
        self.target_layer = self.resnet.layer4[-1]  # Define the target layer

        # Modify the fully connected layer for classification
        self.fc = nn.Linear(2048, num_classes)

    def forward(self, x):
        x = self.resnet.conv1(x)
        x = self.resnet.bn1(x)
        x = self.resnet.relu(x)
        x = self.resnet.maxpool(x)
        x = self.resnet.layer1(x)
        x = self.resnet.layer2(x)
        x = self.resnet.layer3(x)
        x = self.resnet.layer4(x)
        x = self.resnet.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

# Data preprocessing and loading
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Assume you have predefined data loaders (train_loader, val_loader, test_loader)

# Create and train your model
num_classes = 4  # Adjust this according to your dataset
model = MRI_ClassificationModel(num_classes)
torch.cuda.empty_cache()
model.to(device)

criterion = nn.CrossEntropyLoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
num_epochs = 10  # Choose the number of epochs
for epoch in range(num_epochs):
    print(epoch)
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


# Inference and visualization using Grad-CAM
model.eval()
gradcam = GradCAM(model, target_layer=model.resnet.layer4[-1])  # Define target_layer appropriately
target_layer = model.target_layer
for inputs, labels in val_loader:
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    # Use GradCAM to visualize tumor localization
    gradcam_map = generate_gradcam(model, inputs, target_layer)  # Implement generate_gradcam function
    # You can overlay gradcam_map on the input images for visualization

# Evaluate the model's classification accuracy and localization accuracy
# You can use standard evaluation metrics like accuracy, precision, recall, and F1-score.


C:\Users\gabeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\gabeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0
1
2
3
4
5
6
7
8
9


C:\Users\gabeb\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torch\nn\modules\module.py:1359: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


RuntimeError: grad can be implicitly created only for scalar outputs

In [ ]:
import torch
import torch.nn.functional as F



# You need to define a method in your model to get activation gradients from the target layer
# In this example, we assume you have a `get_activation_gradients` method in your model.

class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.target_layer = target_layer
        self.gradients = None

        # Register a hook to capture gradients from the target layer
        self.target_layer.register_forward_hook(self.save_gradients)

    def save_gradients(self, module, input, output):
        self.gradients = output[0]

    def get_gradcam(self, input_image):
        self.model.zero_grad()
        output = self.model(input_image)

        one_hot = torch.zeros_like(output)
        one_hot[0][output.argmax()] = 1

        self.model.zero_grad()
        output.backward(gradient=one_hot, retain_graph=True)

        activations = self.gradients
        weight = F.adaptive_avg_pool2d(activations, 1)
        gradcam_map = (activations * weight).sum(dim=1, keepdim=True)

        gradcam_map = F.relu(gradcam_map)

        return gradcam_map


In [ ]:
import cv2

def localization_accuracy_function(gradcam_map, labels, threshold=0.5):
    """
    Calculate localization accuracy based on the Grad-CAM map.

    Parameters:
    - gradcam_map: Torch tensor containing Grad-CAM maps for each image.
    - labels: True labels for each image.
    - threshold: Threshold value for binarizing the Grad-CAM map.

    Returns:
    - accuracy: Localization accuracy.
    """
    binary_map = (gradcam_map > threshold).float()
    correct_predictions = torch.sum(binary_map * labels)
    total_predictions = torch.sum(binary_map)

    if total_predictions == 0:
        return 0.0

    accuracy = correct_predictions / total_predictions
    return accuracy.item()


def overlay_heatmap(image, heatmap, alpha=0.6):
    """
    Overlay the heatmap on the input image.

    Parameters:
    - image: Input image.
    - heatmap: Grad-CAM heatmap.
    - alpha: Weight for blending the heatmap with the image.

    Returns:
    - overlay: Image with overlaid heatmap.
    """
    heatmap = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(cv2.cvtColor(image, cv2.COLOR_RGB2BGR), 1 - alpha, heatmap, alpha, 0)

    return cv2.cvtColor(overlay, cv2.COLOR_BGR2RGB)


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

model.to(device)
model.eval()

# Lists to store evaluation results
true_labels = []
predicted_labels = []
localization_accuracy = []

num_examples_to_visualize = 5  # Define how many examples you want to visualize

with torch.no_grad():
    for i, (images, labels) in enumerate(val_loader):
        images = images.to(device)
        labels = labels.to(device)

        # Generate Grad-CAM for the images
        gradcam_map = generate_gradcam(model, images, target_layer=model.target_layer)

        # Get the predicted class
        _, predicted_class = model(images).max(1)

        # Calculate classification accuracy
        accuracy = accuracy_score(labels.cpu(), predicted_class.cpu())
        
        # Calculate localization accuracy
        localization_accuracy.append(localization_accuracy_function(gradcam_map, labels, threshold=0.5))
        
        true_labels.extend(labels.cpu().numpy())
        predicted_labels.extend(predicted_class.cpu().numpy())

        if i < num_examples_to_visualize:
            # Visualize the images with Grad-CAM
            for j in range(images.size(0)):
                img = images[j].to(device).numpy().transpose(1, 2, 0)
                heatmap = gradcam_map[j, 0].to(device).numpy()
                heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
                heatmap = (heatmap - np.min(heatmap)) / (np.max(heatmap) - np.min(heatmap))  # Normalize the heatmap
                overlay = overlay_heatmap(img, heatmap)

                plt.figure(figsize=(8, 8))
                plt.subplot(num_examples_to_visualize, images.size(0), i * images.size(0) + j + 1)
                plt.title(f'True: {labels[j].item()}, Predicted: {predicted_class[j].item()}')
                plt.imshow(overlay)
                plt.axis('off')

# Calculate other metrics (e.g., precision, recall, F1-score) using true_labels and predicted_labels

# Calculate mean localization accuracy
mean_localization_accuracy = sum(localization_accuracy) / len(localization_accuracy)

# Print and/or log the evaluation metrics
print(f'Classification Accuracy: {accuracy}')
print(f'Mean Localization Accuracy: {mean_localization_accuracy}')
plt.show()


AttributeError: 'MRI_ClassificationModel' object has no attribute 'target_layer'